# Apache Spark : Analyse des données climatiques mondiales

## Objectif
Analyser les tendances climatiques mondiales à l'aide de Spark, y compris le nettoyage des données, l'EDA et l'extraction d'informations.

### Jeu de données
**Global Surface Summary of the Day (GSOD)** provenant de NOAA.
Source: https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/

---
## 1. Téléchargement des données

Les données GSOD sont disponibles par année. Chaque archive tar.gz contient des fichiers CSV pour toutes les stations.

In [ ]:
import os
import tarfile
import requests
from pathlib import Path

# Créer le répertoire pour les données
data_dir = Path('/home/jovyan/work/data')
data_dir.mkdir(exist_ok=True)

# Liste des années à télécharger (ajustez selon vos besoins)
years = [2019, 2020, 2021, 2022, 2023]

def download_year(year):
    """Télécharge et extrait les données pour une année donnée."""
    url = f"https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/{year}.tar.gz"
    tar_path = data_dir / f"{year}.tar.gz"
    extract_dir = data_dir / str(year)
    
    # Télécharger si pas déjà fait
    if not tar_path.exists():
        print(f"Téléchargement de {year}...")
        response = requests.get(url, stream=True)
        with open(tar_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"✓ {year} téléchargé")
    
    # Extraire
    if not extract_dir.exists():
        print(f"Extraction de {year}...")
        with tarfile.open(tar_path, 'r:gz') as tar:
            tar.extractall(path=extract_dir)
        print(f"✓ {year} extrait")
    
    return extract_dir

# Télécharger les données (décommentez pour exécuter)
# for year in years:
#     download_year(year)

print("\n⚠️ NOTE: Si le téléchargement ne fonctionne pas dans le notebook, vous pouvez:")
print("1. Télécharger manuellement depuis: https://www.ncei.noaa.gov/data/global-summary-of-the-day/archive/")
print("2. Placer les fichiers .tar.gz dans /home/jovyan/work/data/")
print("3. Extraire avec: tar -xzf ANNÉE.tar.gz -C /home/jovyan/work/data/ANNÉE/")

---
## 2. Initialisation de Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Créer la session Spark en mode local
spark = SparkSession.builder \
    .appName("Analyse Climatique GSOD") \
    .master("local[*]") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

# Configurer le niveau de log
spark.sparkContext.setLogLevel("WARN")

print(f"Spark version: {spark.version}")
print(f"✓ Session Spark initialisée")

---
## 3. Chargement et exploration des données

Les fichiers GSOD contiennent les colonnes suivantes:
- STATION: Identifiant de la station
- DATE: Date de l'observation
- LATITUDE, LONGITUDE: Coordonnées
- ELEVATION: Altitude
- NAME: Nom de la station
- TEMP: Température moyenne (°F)
- TEMP_ATTRIBUTES: Attributs température
- DEWP: Point de rosée
- SLP: Pression au niveau de la mer
- STP: Pression à la station
- VISIB: Visibilité
- WDSP: Vitesse du vent
- MXSPD: Vitesse maximale du vent
- GUST: Rafale
- MAX: Température maximale
- MIN: Température minimale
- PRCP: Précipitations
- SNDP: Profondeur de neige
- FRSHTT: Indicateurs météo (brouillard, pluie, neige, grêle, orage, tornade)

In [ ]:
# Définir le schéma pour les données GSOD
gsod_schema = StructType([
    StructField("STATION", StringType(), True),
    StructField("DATE", StringType(), True),
    StructField("LATITUDE", DoubleType(), True),
    StructField("LONGITUDE", DoubleType(), True),
    StructField("ELEVATION", DoubleType(), True),
    StructField("NAME", StringType(), True),
    StructField("TEMP", DoubleType(), True),
    StructField("TEMP_ATTRIBUTES", IntegerType(), True),
    StructField("DEWP", DoubleType(), True),
    StructField("DEWP_ATTRIBUTES", IntegerType(), True),
    StructField("SLP", DoubleType(), True),
    StructField("SLP_ATTRIBUTES", IntegerType(), True),
    StructField("STP", DoubleType(), True),
    StructField("STP_ATTRIBUTES", IntegerType(), True),
    StructField("VISIB", DoubleType(), True),
    StructField("VISIB_ATTRIBUTES", IntegerType(), True),
    StructField("WDSP", DoubleType(), True),
    StructField("WDSP_ATTRIBUTES", IntegerType(), True),
    StructField("MXSPD", DoubleType(), True),
    StructField("GUST", DoubleType(), True),
    StructField("MAX", DoubleType(), True),
    StructField("MAX_ATTRIBUTES", StringType(), True),
    StructField("MIN", DoubleType(), True),
    StructField("MIN_ATTRIBUTES", StringType(), True),
    StructField("PRCP", DoubleType(), True),
    StructField("PRCP_ATTRIBUTES", StringType(), True),
    StructField("SNDP", DoubleType(), True),
    StructField("FRSHTT", StringType(), True)
])

In [ ]:
# Charger tous les fichiers CSV
# Ajustez le chemin selon votre structure de répertoires
data_path = "/home/jovyan/work/data/*/*.csv"

df = spark.read.csv(
    data_path,
    header=True,
    schema=gsod_schema
)

print(f"Nombre total d'enregistrements: {df.count():,}")
print(f"Nombre de partitions: {df.rdd.getNumPartitions()}")

# Afficher le schéma
df.printSchema()

In [ ]:
# Afficher quelques enregistrements
df.show(5, truncate=False)

In [ ]:
# Statistiques descriptives
df.select("TEMP", "PRCP", "MAX", "MIN").describe().show()

---
## 4. Nettoyage des données

Les valeurs manquantes ou invalides dans GSOD sont souvent codées comme 9999.9

In [ ]:
# Remplacer les valeurs 9999.9 par null et créer l'année
df_clean = df \
    .withColumn("TEMP", when(col("TEMP") == 9999.9, None).otherwise(col("TEMP"))) \
    .withColumn("PRCP", when(col("PRCP") == 99.99, None).otherwise(col("PRCP"))) \
    .withColumn("MAX", when(col("MAX") == 9999.9, None).otherwise(col("MAX"))) \
    .withColumn("MIN", when(col("MIN") == 9999.9, None).otherwise(col("MIN"))) \
    .withColumn("year", year(col("DATE"))) \
    .withColumn("month", month(col("DATE"))) \
    .withColumn("TEMP_C", (col("TEMP") - 32) * 5.0 / 9.0) \
    .withColumn("MAX_C", (col("MAX") - 32) * 5.0 / 9.0) \
    .withColumn("MIN_C", (col("MIN") - 32) * 5.0 / 9.0) \
    .withColumn("PRCP_MM", col("PRCP") * 25.4)

# Filtrer les enregistrements avec des températures valides
df_clean = df_clean.filter(col("TEMP").isNotNull())

print(f"Enregistrements après nettoyage: {df_clean.count():,}")

# Mettre en cache pour optimiser les requêtes suivantes
df_clean.cache()
df_clean.count()  # Trigger cache

print("✓ Données nettoyées et mises en cache")

In [ ]:
# Vérifier les valeurs manquantes
from pyspark.sql.functions import count, when, isnan, col

df_clean.select(
    [count(when(col(c).isNull(), c)).alias(c) for c in ["TEMP_C", "PRCP_MM", "MAX_C", "MIN_C"]]
).show()

---
## 5. Analyse avec Spark SQL

Créons une vue temporaire pour utiliser SQL

In [ ]:
# Créer une vue temporaire
df_clean.createOrReplaceTempView("climate_data")

print("✓ Vue 'climate_data' créée")

### Question 1: Quelle a été l'année la plus froide enregistrée, et quelle était la température moyenne ?

In [ ]:
# Requête SQL pour trouver l'année la plus froide
coldest_year_sql = """
SELECT 
    year,
    ROUND(AVG(TEMP_C), 2) as avg_temperature_c,
    COUNT(*) as num_records
FROM climate_data
WHERE TEMP_C IS NOT NULL
GROUP BY year
ORDER BY avg_temperature_c ASC
LIMIT 1
"""

coldest_year = spark.sql(coldest_year_sql)
coldest_year.show()

# Avec DataFrame API
coldest_year_df = df_clean \
    .groupBy("year") \
    .agg(
        round(avg("TEMP_C"), 2).alias("avg_temperature_c"),
        count("*").alias("num_records")
    ) \
    .orderBy("avg_temperature_c") \
    .limit(1)

print("\nAvec DataFrame API:")
coldest_year_df.show()

### Question 2: Quelle station a contribué avec le plus grand nombre d'enregistrements ?

In [ ]:
# Requête SQL pour trouver la station la plus active
top_station_sql = """
SELECT 
    STATION,
    NAME,
    LATITUDE,
    LONGITUDE,
    COUNT(*) as num_records
FROM climate_data
GROUP BY STATION, NAME, LATITUDE, LONGITUDE
ORDER BY num_records DESC
LIMIT 10
"""

top_stations = spark.sql(top_station_sql)
top_stations.show(10, truncate=False)

# Avec DataFrame API
top_stations_df = df_clean \
    .groupBy("STATION", "NAME", "LATITUDE", "LONGITUDE") \
    .agg(count("*").alias("num_records")) \
    .orderBy(col("num_records").desc()) \
    .limit(10)

print("\nAvec DataFrame API:")
top_stations_df.show(10, truncate=False)

### Analyses supplémentaires

In [ ]:
# Tendances annuelles de température
yearly_stats = spark.sql("""
SELECT 
    year,
    ROUND(AVG(TEMP_C), 2) as avg_temp_c,
    ROUND(AVG(MAX_C), 2) as avg_max_c,
    ROUND(AVG(MIN_C), 2) as avg_min_c,
    ROUND(AVG(PRCP_MM), 2) as avg_prcp_mm,
    COUNT(*) as num_records
FROM climate_data
WHERE TEMP_C IS NOT NULL
GROUP BY year
ORDER BY year
""")

yearly_stats.show()

In [ ]:
# Patterns mensuels
monthly_patterns = spark.sql("""
SELECT 
    month,
    ROUND(AVG(TEMP_C), 2) as avg_temp_c,
    ROUND(AVG(PRCP_MM), 2) as avg_prcp_mm,
    COUNT(*) as num_records
FROM climate_data
WHERE TEMP_C IS NOT NULL
GROUP BY month
ORDER BY month
""")

monthly_patterns.show(12)

---
## 6. Visualisations

### Question 1: Graphe des températures moyennes mondiales

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Configuration du style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)

# Convertir en Pandas pour la visualisation
yearly_stats_pd = yearly_stats.toPandas()

# Graphique des températures
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Température moyenne par année
ax1.plot(yearly_stats_pd['year'], yearly_stats_pd['avg_temp_c'], 
         marker='o', linewidth=2, markersize=8, color='#e74c3c')
ax1.fill_between(yearly_stats_pd['year'], 
                  yearly_stats_pd['avg_min_c'], 
                  yearly_stats_pd['avg_max_c'], 
                  alpha=0.3, color='#e74c3c')
ax1.set_xlabel('Année', fontsize=12)
ax1.set_ylabel('Température (°C)', fontsize=12)
ax1.set_title('Évolution des températures moyennes mondiales', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Température par mois (pattern saisonnier)
monthly_patterns_pd = monthly_patterns.toPandas()
months = ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun', 'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc']
ax2.bar(monthly_patterns_pd['month'], monthly_patterns_pd['avg_temp_c'], 
        color='#3498db', alpha=0.7)
ax2.set_xlabel('Mois', fontsize=12)
ax2.set_ylabel('Température moyenne (°C)', fontsize=12)
ax2.set_title('Pattern saisonnier des températures', fontsize=14, fontweight='bold')
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels(months)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/home/jovyan/work/temperature_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Graphique des températures sauvegardé: temperature_trends.png")

### Question 2: Graphe des précipitations mondiales

In [ ]:
# Graphique des précipitations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Précipitations par année
ax1.plot(yearly_stats_pd['year'], yearly_stats_pd['avg_prcp_mm'], 
         marker='s', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_xlabel('Année', fontsize=12)
ax1.set_ylabel('Précipitations moyennes (mm)', fontsize=12)
ax1.set_title('Évolution des précipitations moyennes mondiales', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Précipitations par mois
ax2.bar(monthly_patterns_pd['month'], monthly_patterns_pd['avg_prcp_mm'], 
        color='#1abc9c', alpha=0.7)
ax2.set_xlabel('Mois', fontsize=12)
ax2.set_ylabel('Précipitations moyennes (mm)', fontsize=12)
ax2.set_title('Pattern saisonnier des précipitations', fontsize=14, fontweight='bold')
ax2.set_xticks(range(1, 13))
ax2.set_xticklabels(months)
ax2.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/home/jovyan/work/precipitation_trends.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Graphique des précipitations sauvegardé: precipitation_trends.png")

### Question 3: Analyses supplémentaires - Réchauffement climatique

In [ ]:
# Calculer la tendance du réchauffement
from scipy import stats

# Régression linéaire pour la tendance
slope, intercept, r_value, p_value, std_err = stats.linregress(
    yearly_stats_pd['year'], 
    yearly_stats_pd['avg_temp_c']
)

# Créer la ligne de tendance
trend_line = slope * yearly_stats_pd['year'] + intercept

# Visualisation de la tendance
plt.figure(figsize=(14, 8))

plt.subplot(2, 1, 1)
plt.plot(yearly_stats_pd['year'], yearly_stats_pd['avg_temp_c'], 
         marker='o', linewidth=2, markersize=8, label='Température observée', color='#e74c3c')
plt.plot(yearly_stats_pd['year'], trend_line, 
         '--', linewidth=2, label=f'Tendance ({slope:.4f}°C/an)', color='#c0392b')
plt.xlabel('Année', fontsize=12)
plt.ylabel('Température (°C)', fontsize=12)
plt.title('Tendance du réchauffement climatique', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Anomalie par rapport à la moyenne
plt.subplot(2, 1, 2)
mean_temp = yearly_stats_pd['avg_temp_c'].mean()
anomaly = yearly_stats_pd['avg_temp_c'] - mean_temp
colors = ['#e74c3c' if x > 0 else '#3498db' for x in anomaly]
plt.bar(yearly_stats_pd['year'], anomaly, color=colors, alpha=0.7)
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
plt.xlabel('Année', fontsize=12)
plt.ylabel('Anomalie de température (°C)', fontsize=12)
plt.title(f'Anomalie de température (Référence: {mean_temp:.2f}°C)', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.savefig('/home/jovyan/work/warming_trend.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n📊 Statistiques de tendance:")
print(f"   Taux de réchauffement: {slope:.4f}°C par an")
print(f"   R²: {r_value**2:.4f}")
print(f"   P-value: {p_value:.4e}")
print(f"   Augmentation totale sur la période: {slope * len(yearly_stats_pd):.2f}°C")

if p_value < 0.05:
    print("   ✓ La tendance est statistiquement significative (p < 0.05)")
else:
    print("   ⚠ La tendance n'est pas statistiquement significative")

### Question 4: Augmentation de température par pays

In [ ]:
# Extraire le pays du nom de la station (simplifié - basé sur les deux dernières lettres)
# Note: Une approche plus robuste utiliserait un dataset de géolocalisation

# Calculer la température moyenne globale sur toute la période
global_avg_temp = df_clean.select(avg("TEMP_C")).collect()[0][0]

print(f"Température moyenne mondiale: {global_avg_temp:.2f}°C")

# Analyser par région géographique (basé sur la latitude)
df_regions = df_clean.withColumn(
    "region",
    when(col("LATITUDE") >= 60, "Arctique/Subarctique")
    .when((col("LATITUDE") >= 30) & (col("LATITUDE") < 60), "Zone tempérée Nord")
    .when((col("LATITUDE") >= 0) & (col("LATITUDE") < 30), "Zone tropicale Nord")
    .when((col("LATITUDE") >= -30) & (col("LATITUDE") < 0), "Zone tropicale Sud")
    .when((col("LATITUDE") >= -60) & (col("LATITUDE") < -30), "Zone tempérée Sud")
    .otherwise("Antarctique/Subantarctique")
)

# Calculer l'augmentation par région
regional_temps = df_regions.groupBy("region", "year") \
    .agg(avg("TEMP_C").alias("avg_temp"))

# Calculer la moyenne par région
regional_avg = df_regions.groupBy("region") \
    .agg(
        avg("TEMP_C").alias("avg_temp_c"),
        count("*").alias("num_records")
    ) \
    .withColumn(
        "pct_difference_from_global",
        ((col("avg_temp_c") - global_avg_temp) / global_avg_temp * 100)
    ) \
    .orderBy(col("pct_difference_from_global").desc())

regional_avg_pd = regional_avg.toPandas()
regional_avg_pd

In [ ]:
# Visualisation par région
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Température moyenne par région
colors_regional = plt.cm.RdYlBu_r(np.linspace(0, 1, len(regional_avg_pd)))
ax1.barh(regional_avg_pd['region'], regional_avg_pd['avg_temp_c'], color=colors_regional)
ax1.axvline(x=global_avg_temp, color='red', linestyle='--', linewidth=2, label=f'Moyenne mondiale: {global_avg_temp:.2f}°C')
ax1.set_xlabel('Température moyenne (°C)', fontsize=12)
ax1.set_title('Température moyenne par région', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3, axis='x')

# Pourcentage de différence par rapport à la moyenne mondiale
colors_pct = ['#e74c3c' if x > 0 else '#3498db' for x in regional_avg_pd['pct_difference_from_global']]
ax2.barh(regional_avg_pd['region'], regional_avg_pd['pct_difference_from_global'], color=colors_pct, alpha=0.7)
ax2.axvline(x=0, color='black', linestyle='-', linewidth=0.8)
ax2.set_xlabel('Différence par rapport à la moyenne mondiale (%)', fontsize=12)
ax2.set_title('Écart de température par région', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='x')

# Ajouter les valeurs sur les barres
for i, (idx, row) in enumerate(regional_avg_pd.iterrows()):
    ax2.text(row['pct_difference_from_global'], i, f" {row['pct_difference_from_global']:.1f}%", 
             va='center', fontsize=10)

plt.tight_layout()
plt.savefig('/home/jovyan/work/regional_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Graphique régional sauvegardé: regional_analysis.png")

---
## 7. Conclusions et analyses supplémentaires recommandées

### Analyses recommandées:

1. **Analyse temporelle avancée:**
   - Décomposition saisonnière (trend, saisonnalité, résidus)
   - Détection de points de rupture dans les séries temporelles
   - Prévisions avec modèles ARIMA/Prophet

2. **Analyse spatiale:**
   - Cartographie des hotspots de réchauffement
   - Analyse de corrélation spatiale (Moran's I)
   - Identification des zones à risque élevé

3. **Analyse des événements extrêmes:**
   - Fréquence et intensité des vagues de chaleur
   - Événements de précipitations extrêmes
   - Périodes de sécheresse prolongée

4. **Corrélations avec d'autres facteurs:**
   - Altitude et température
   - Proximité océanique et variabilité climatique
   - Latitude et amplitude thermique

5. **Machine Learning:**
   - Prédiction des températures futures
   - Classification des régimes climatiques
   - Détection d'anomalies

### Isolation des effets du réchauffement climatique:

Pour isoler les effets du réchauffement:
- Comparer les données récentes (2020-2023) avec des périodes de référence (1980-2000)
- Analyser les tendances à long terme (plusieurs décennies)
- Utiliser des modèles de régression pour contrôler les variations naturelles
- Comparer avec les modèles climatiques du GIEC
- Analyser les indicateurs secondaires (fonte des glaces, élévation du niveau de la mer, etc.)

In [ ]:
# Analyse des événements extrêmes
extreme_events = spark.sql("""
SELECT 
    year,
    COUNT(CASE WHEN MAX_C > 40 THEN 1 END) as days_above_40c,
    COUNT(CASE WHEN MIN_C < -20 THEN 1 END) as days_below_minus20c,
    COUNT(CASE WHEN PRCP_MM > 100 THEN 1 END) as heavy_rainfall_days,
    MAX(MAX_C) as max_temp_recorded,
    MIN(MIN_C) as min_temp_recorded
FROM climate_data
WHERE MAX_C IS NOT NULL AND MIN_C IS NOT NULL
GROUP BY year
ORDER BY year
""")

extreme_events.show()

In [ ]:
# Visualisation des événements extrêmes
extreme_pd = extreme_events.toPandas()

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Jours > 40°C
ax1.bar(extreme_pd['year'], extreme_pd['days_above_40c'], color='#e74c3c', alpha=0.7)
ax1.set_title('Jours avec températures > 40°C', fontsize=12, fontweight='bold')
ax1.set_ylabel('Nombre de jours', fontsize=11)
ax1.grid(True, alpha=0.3, axis='y')

# Jours < -20°C
ax2.bar(extreme_pd['year'], extreme_pd['days_below_minus20c'], color='#3498db', alpha=0.7)
ax2.set_title('Jours avec températures < -20°C', fontsize=12, fontweight='bold')
ax2.set_ylabel('Nombre de jours', fontsize=11)
ax2.grid(True, alpha=0.3, axis='y')

# Jours de fortes précipitations
ax3.bar(extreme_pd['year'], extreme_pd['heavy_rainfall_days'], color='#2ecc71', alpha=0.7)
ax3.set_title('Jours avec précipitations > 100mm', fontsize=12, fontweight='bold')
ax3.set_ylabel('Nombre de jours', fontsize=11)
ax3.set_xlabel('Année', fontsize=11)
ax3.grid(True, alpha=0.3, axis='y')

# Températures record
ax4.plot(extreme_pd['year'], extreme_pd['max_temp_recorded'], 
         marker='o', label='Max record', color='#e74c3c', linewidth=2)
ax4.plot(extreme_pd['year'], extreme_pd['min_temp_recorded'], 
         marker='s', label='Min record', color='#3498db', linewidth=2)
ax4.set_title('Températures record par année', fontsize=12, fontweight='bold')
ax4.set_ylabel('Température (°C)', fontsize=11)
ax4.set_xlabel('Année', fontsize=11)
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/home/jovyan/work/extreme_events.png', dpi=300, bbox_inches='tight')
plt.show()

print("✓ Graphique des événements extrêmes sauvegardé: extreme_events.png")

---
## 8. Résumé des requêtes SQL utilisées

In [ ]:
print("""
=== REQUÊTES SQL PRINCIPALES ===

1. ANNÉE LA PLUS FROIDE:
SELECT year, ROUND(AVG(TEMP_C), 2) as avg_temperature_c, COUNT(*) as num_records
FROM climate_data
WHERE TEMP_C IS NOT NULL
GROUP BY year
ORDER BY avg_temperature_c ASC
LIMIT 1

2. STATION LA PLUS ACTIVE:
SELECT STATION, NAME, LATITUDE, LONGITUDE, COUNT(*) as num_records
FROM climate_data
GROUP BY STATION, NAME, LATITUDE, LONGITUDE
ORDER BY num_records DESC
LIMIT 10

3. TENDANCES ANNUELLES:
SELECT year, 
       ROUND(AVG(TEMP_C), 2) as avg_temp_c,
       ROUND(AVG(MAX_C), 2) as avg_max_c,
       ROUND(AVG(MIN_C), 2) as avg_min_c,
       ROUND(AVG(PRCP_MM), 2) as avg_prcp_mm,
       COUNT(*) as num_records
FROM climate_data
WHERE TEMP_C IS NOT NULL
GROUP BY year
ORDER BY year

4. PATTERNS MENSUELS:
SELECT month,
       ROUND(AVG(TEMP_C), 2) as avg_temp_c,
       ROUND(AVG(PRCP_MM), 2) as avg_prcp_mm,
       COUNT(*) as num_records
FROM climate_data
WHERE TEMP_C IS NOT NULL
GROUP BY month
ORDER BY month

5. ÉVÉNEMENTS EXTRÊMES:
SELECT year,
       COUNT(CASE WHEN MAX_C > 40 THEN 1 END) as days_above_40c,
       COUNT(CASE WHEN MIN_C < -20 THEN 1 END) as days_below_minus20c,
       COUNT(CASE WHEN PRCP_MM > 100 THEN 1 END) as heavy_rainfall_days,
       MAX(MAX_C) as max_temp_recorded,
       MIN(MIN_C) as min_temp_recorded
FROM climate_data
WHERE MAX_C IS NOT NULL AND MIN_C IS NOT NULL
GROUP BY year
ORDER BY year
""")

---
## 9. Nettoyage et fermeture

In [ ]:
# Exporter les résultats principaux
yearly_stats.coalesce(1).write.mode("overwrite").csv(
    "/home/jovyan/work/results/yearly_statistics", 
    header=True
)

regional_avg.coalesce(1).write.mode("overwrite").csv(
    "/home/jovyan/work/results/regional_analysis", 
    header=True
)

print("✓ Résultats exportés dans /home/jovyan/work/results/")

In [ ]:
# Fermer la session Spark
# spark.stop()
# print("✓ Session Spark fermée")

---
## Notes finales

### Performance et optimisation:
- Les données ont été mises en cache pour accélérer les requêtes répétées
- Utilisation de `coalesce()` pour réduire le nombre de partitions lors des exports
- Les conversions vers Pandas sont limitées aux résultats agrégés pour la visualisation

### Points d'amélioration possibles:
1. Intégration avec HDFS pour le stockage distribué
2. Utilisation de Spark Structured Streaming pour l'analyse en temps réel
3. Intégration avec MLlib pour des prédictions avancées
4. Dashboard interactif avec Plotly Dash ou Streamlit
5. Pipeline ETL automatisé avec Airflow

### Ressources supplémentaires:
- Documentation GSOD: https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/
- Spark SQL Guide: https://spark.apache.org/docs/latest/sql-programming-guide.html
- Climate Data Analysis: https://www.ipcc.ch/

---
**Auteur:** Analyse climatique avec Apache Spark  
**Date:** 2024  
**Version:** 1.0
"""